# Segmenting and Clustering Neighborhoods in Toronto

For this assignment, you will be required to explore and cluster the neighborhoods in Toronto.

Start by creating a new Notebook for this assignment. Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe like the one shown below.

# Importing Libraries

In [9]:
import pandas as pd
import numpy as np

#json tools
import json
from pandas.io.json import json_normalize

#scraping
import requests
from urllib.request import urlopen
from bs4 import BeautifulSoup

#geocoders
from geopy.geocoders import Nominatim

#visualization libraries
import matplotlib.cm as cm
import matplotlib.colors as colors
import folium

#kmeans clustering
from sklearn.cluster import KMeans

# Task 1 - Scraping Wikipedia page, creating Pandas DF, cleaning data

Using BeautifulSoup and URLopen libraries

In [3]:
wlink = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
raw_page = urlopen(wlink).read().decode('utf-8')
page = BeautifulSoup(raw_page, 'html.parser')
table = page.body.table.tbody

In [4]:
#functions for getting cell and row data

def table_cell(i):
    cells = i.find_all('td')
    row = []
    
    for cell in cells:
        if cell.a:            
            if (cell.a.text):
                row.append(cell.a.text)
                continue
        row.append(cell.string.strip())
        
    return row

def table_row():    
    data = []  
    
    for tr in table.find_all('tr'):
        row = table_cell(tr)
        if len(row) != 3:
            continue
        data.append(row)        
    
    return data

In [5]:
data = table_row()
columns = ['Postcode', 'Borough', 'Neighbourhood']
df = pd.DataFrame(data, columns=columns)
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [7]:
#dropping the "Not Assigned" borough
df1 = df[df.Borough != 'Not assigned']
df1 = df1.sort_values(by=['Postcode','Borough'])

df1.reset_index(inplace=True)
df1.drop('index',axis=1,inplace=True)
df1.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,Rouge
1,M1B,Scarborough,Malvern
2,M1C,Scarborough,Highland Creek
3,M1C,Scarborough,Rouge Hill
4,M1C,Scarborough,Port Union


In [11]:
#Consolidating the neighbourhoods that share the postcode

df_postcodes = df1['Postcode']
df_postcodes.drop_duplicates(inplace=True)
df2 = pd.DataFrame(df_postcodes)
df2['Borough'] = '';
df2['Neighbourhood'] = '';


df2.reset_index(inplace=True)
df2.drop('index', axis=1, inplace=True)
df1.reset_index(inplace=True)
df1.drop('index', axis=1, inplace=True)

for i in df2.index:
    for j in df1.index:
        if df2.iloc[i, 0] == df1.iloc[j, 0]:
            df2.iloc[i, 1] = df1.iloc[j, 1]
            df2.iloc[i, 2] = df2.iloc[i, 2] + ',' + df1.iloc[j, 2]
            
for i in df2.index:
    s = df2.iloc[i, 2]
    if s[0] == ',':
        s =s [1:]
    df2.iloc[i,2 ] = s
    
df2.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


# Task 2 - Get Coordinates

Now that you have built a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name, in order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood.

Using the provided Geospatial_Coordinates.csv file to get the coordinates:

In [13]:
#reading the file to coord dataframe
df2['Latitude'] = '0';
df2['Longitude'] = '0';

coord = pd.read_csv('https://cocl.us/Geospatial_data')

In [14]:
#merging dataframe that contain coordinates with the one that contains borough names
for i in df2.index:
    for j in coord.index:
        if df2.iloc[i, 0] == coord.iloc[j, 0]:
            df2.iloc[i, 3] = coord.iloc[j, 1]
            df2.iloc[i, 4] = coord.iloc[j, 2]

#checking the results            
df2.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.8067,-79.1944
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.7845,-79.1605
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.7636,-79.1887
3,M1G,Scarborough,Woburn,43.771,-79.2169
4,M1H,Scarborough,Cedarbrae,43.7731,-79.2395


# Task 3 - Analysis

Explore and cluster the neighborhoods in Toronto. You can decide to work with only boroughs that contain the word Toronto and then replicate the same analysis we did to the New York City data. It is up to you.

Just make sure:

to add enough Markdown cells to explain what you decided to do and to report any observations you make.
to generate maps to visualize your neighborhoods and how they cluster together.

# 3.1 Select only the neighbourhoods of Downtown Toronto

Choose the neighbourhoods that contain word " Downtown Toronto":

In [17]:
toronto = df2[df2['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
toronto.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M4W,Downtown Toronto,Rosedale,43.6796,-79.3775
1,M4X,Downtown Toronto,"Cabbagetown,St. James Town",43.668,-79.3677
2,M4Y,Downtown Toronto,Church and Wellesley,43.6659,-79.3832
3,M5A,Downtown Toronto,Harbourfront,43.6543,-79.3606
4,M5B,Downtown Toronto,"Ryerson,Garden District",43.6572,-79.3789


In [31]:
#get the coordinates for Toronto
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Toronto are 43.653963, -79.387207.


In [29]:
#create the Folium map of Downtown Toronto
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=13)

# add markers to map
for lat, lng, label in zip(toronto['Latitude'], toronto['Longitude'], toronto['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

# 3.2 Utilizing the Foursquare API to get top 100 venues in Downtown Toronto

In [30]:
#set credintials
CLIENT_ID = 'KXB5T2AHBQITZHDAMVUPAA0E3F1TLGMVMII2OQFFY5HRU0MM' # your Foursquare ID
CLIENT_SECRET = 'KHVMAHSAW5TK0ZIBFVOSJXZ2KQ2FKZOOXHQYLHFQL3CVYMRL' # your Foursquare Secret
VERSION = '20190323' # Foursquare API version

Borrowing the function from the lab to get Top 100 venues in Downtown Toronto within a radius of 500m:

In [32]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Now write the code to run the above function on each neighborhood and create a new dataframe called manhattan_venues.

In [33]:
LIMIT = 100 # limit of number of venues returned by Foursquare API

downtown_venues = getNearbyVenues(names=toronto['Neighbourhood'],
                                   latitudes=toronto['Latitude'],
                                   longitudes=toronto['Longitude']
                                  )

Rosedale
Cabbagetown,St. James Town
Church and Wellesley
Harbourfront
Ryerson,Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide,King,Richmond
Harbourfront East,Toronto Islands,Union Station
Design Exchange,Toronto Dominion Centre
Commerce Court,Victoria Hotel
Harbord,University of Toronto
Chinatown,Grange Park,Kensington Market
CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place,Underground city
Christie
Queen's Park


In [34]:
#checking the size of venues dataframe
downtown_venues.shape

(1318, 7)

In [35]:
#checking how many unique categories of venues are there
print('There are {} unique categories.'.format(len(downtown_venues['Venue Category'].unique())))

There are 205 unique categories.


# 3.3 Analyze each neighbourhood

In [36]:
# one hot encoding
dt_onehot = pd.get_dummies(downtown_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
dt_onehot['Neighborhood'] = downtown_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [dt_onehot.columns[-1]] + list(dt_onehot.columns[:-1])
dt_onehot = dt_onehot[fixed_columns]

dt_onehot.head()

,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [37]:
#checking the dataframe size
dt_onehot.shape

(1318, 205)

Gouping rows by neighborhood and by taking the mean of the frequency of occurrence of each category:

In [38]:
dt_grouped = dt_onehot.groupby('Neighborhood').mean().reset_index()
dt_grouped

,Neighborhood,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store
0,"Adelaide,King,Richmond",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020000,0.000000,...,0.000000,0.00,0.00,0.00,0.020000,0.000000,0.000000,0.010000,0.000000,0.01
1,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,0.00,0.00,0.017544,0.000000,0.000000,0.000000,0.000000,0.00
2,"CN Tower,Bathurst Quay,Island airport,Harbourf...",0.000000,0.000000,0.058824,0.058824,0.117647,0.176471,0.117647,0.000000,0.000000,...,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
3,"Cabbagetown,St. James Town",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020408,0.000000,...,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
4,Central Bay Street,0.012658,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.012658,0.000000,...,0.000000,0.00,0.00,0.00,0.012658,0.000000,0.000000,0.012658,0.000000,0.00
5,"Chinatown,Grange Park,Kensington Market",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,0.00,0.00,0.034483,0.000000,0.057471,0.011494,0.000000,0.00
6,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
7,Church and Wellesley,0.011628,0.011628,0.000000,0.000000,0.000000,0.000000,0.000000,0.011628,0.000000,...,0.011628,0.00,0.00,0.00,0.000000,0.000000,0.011628,0.000000,0.011628,0.00
8,"Commerce Court,Victoria Hotel",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.040000,0.000000,...,0.000000,0.00,0.00,0.00,0.020000,0.000000,0.000000,0.010000,0.000000,0.00
9,"Design Exchange,Toronto Dominion Centre",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.030000,0.000000,...,0.000000,0.00,0.00,0.01,0.010000,0.000000,0.000000,0.010000,0.000000,0.00


In [39]:
#checking the grouped dataframe size
dt_grouped.shape

(19, 205)

In [40]:
num_top_venues = 5

for hood in dt_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = dt_grouped[dt_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide,King,Richmond----
             venue  freq
0      Coffee Shop  0.07
1             Café  0.04
2  Thai Restaurant  0.04
3              Bar  0.04
4       Restaurant  0.04


----Berczy Park----
                venue  freq
0         Coffee Shop  0.07
1      Farmers Market  0.04
2            Beer Bar  0.04
3                Café  0.04
4  Seafood Restaurant  0.04


----CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara----
              venue  freq
0   Airport Service  0.18
1    Airport Lounge  0.12
2  Airport Terminal  0.12
3             Plane  0.06
4       Coffee Shop  0.06


----Cabbagetown,St. James Town----
         venue  freq
0  Coffee Shop  0.08
1  Pizza Place  0.06
2       Bakery  0.06
3         Café  0.04
4     Pharmacy  0.04


----Central Bay Street----
                venue  freq
0         Coffee Shop  0.16
1  Italian Restaurant  0.05
2      Sandwich Place  0.05
3        Burger Joint  0.04
4      Ice Cream Shop  0.04




In [41]:
#function to sort the venues in descending order:

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [42]:
#create the new dataframe and display the top 10 venues for each neighborhood:

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = dt_grouped['Neighborhood']

for ind in np.arange(dt_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(dt_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Coffee Shop,Café,Restaurant,Bar,Thai Restaurant,Sushi Restaurant,Seafood Restaurant,Gastropub,Lounge,Cosmetics Shop
1,Berczy Park,Coffee Shop,Cheese Shop,Seafood Restaurant,Café,Restaurant,Cocktail Bar,Bakery,Farmers Market,French Restaurant,Beer Bar
2,"CN Tower,Bathurst Quay,Island airport,Harbourf...",Airport Service,Airport Lounge,Airport Terminal,Boat or Ferry,Harbor / Marina,Sculpture Garden,Rental Car Location,Bar,Coffee Shop,Plane
3,"Cabbagetown,St. James Town",Coffee Shop,Bakery,Pizza Place,Café,Italian Restaurant,Restaurant,Pharmacy,Pub,Chinese Restaurant,Indian Restaurant
4,Central Bay Street,Coffee Shop,Sandwich Place,Italian Restaurant,Juice Bar,Café,Burger Joint,Ice Cream Shop,Japanese Restaurant,Salad Place,Bubble Tea Shop


# 3.4 Cluster Neighborhoods

Run k-means to cluster the neighborhood into 5 clusters.

In [43]:
# set number of clusters
kclusters = 4

dt_grouped_clustering = dt_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(dt_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 1, 2, 1, 1, 1, 3, 1, 1, 1])

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [44]:
#add clustering labels
neighborhoods_venues_sorted.insert(0, 'Labels', kmeans.labels_)

dt_merged = toronto

# merge downtown_grouped with toronto data to add latitude/longitude for each neighborhood
# I realized that I've misspelled the NeighboUrhood column name in Toronto dataframe. oops...
dt_merged = dt_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

dt_merged.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4W,Downtown Toronto,Rosedale,43.6796,-79.3775,0,Park,Playground,Trail,Department Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center
1,M4X,Downtown Toronto,"Cabbagetown,St. James Town",43.668,-79.3677,1,Coffee Shop,Bakery,Pizza Place,Café,Italian Restaurant,Restaurant,Pharmacy,Pub,Chinese Restaurant,Indian Restaurant
2,M4Y,Downtown Toronto,Church and Wellesley,43.6659,-79.3832,1,Coffee Shop,Japanese Restaurant,Gay Bar,Sushi Restaurant,Restaurant,Café,Pizza Place,Hotel,Gastropub,Fast Food Restaurant
3,M5A,Downtown Toronto,Harbourfront,43.6543,-79.3606,1,Coffee Shop,Bakery,Park,Café,Pub,Theater,Restaurant,Mexican Restaurant,Breakfast Spot,Beer Store
4,M5B,Downtown Toronto,"Ryerson,Garden District",43.6572,-79.3789,1,Clothing Store,Coffee Shop,Bubble Tea Shop,Café,Middle Eastern Restaurant,Japanese Restaurant,Cosmetics Shop,Italian Restaurant,Ramen Restaurant,Plaza


In [45]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=13)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(dt_merged['Latitude'], dt_merged['Longitude'], dt_merged['Neighbourhood'], dt_merged['Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=9,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# 3.5 Examine the clusters

In [46]:
#Cluster 1
dt_merged.loc[dt_merged['Labels'] == 0, dt_merged.columns[[1] + list(range(5, dt_merged.shape[1]))]]

,Borough,Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,0,Park,Playground,Trail,Department Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center


In [47]:
#Cluster 2
dt_merged.loc[dt_merged['Labels'] == 1, dt_merged.columns[[1] + list(range(5, dt_merged.shape[1]))]]

,Borough,Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Downtown Toronto,1,Coffee Shop,Bakery,Pizza Place,Café,Italian Restaurant,Restaurant,Pharmacy,Pub,Chinese Restaurant,Indian Restaurant
2,Downtown Toronto,1,Coffee Shop,Japanese Restaurant,Gay Bar,Sushi Restaurant,Restaurant,Café,Pizza Place,Hotel,Gastropub,Fast Food Restaurant
3,Downtown Toronto,1,Coffee Shop,Bakery,Park,Café,Pub,Theater,Restaurant,Mexican Restaurant,Breakfast Spot,Beer Store
4,Downtown Toronto,1,Clothing Store,Coffee Shop,Bubble Tea Shop,Café,Middle Eastern Restaurant,Japanese Restaurant,Cosmetics Shop,Italian Restaurant,Ramen Restaurant,Plaza
5,Downtown Toronto,1,Coffee Shop,Café,Restaurant,Clothing Store,Hotel,Diner,Breakfast Spot,Beer Bar,Cosmetics Shop,Bakery
6,Downtown Toronto,1,Coffee Shop,Cheese Shop,Seafood Restaurant,Café,Restaurant,Cocktail Bar,Bakery,Farmers Market,French Restaurant,Beer Bar
7,Downtown Toronto,1,Coffee Shop,Sandwich Place,Italian Restaurant,Juice Bar,Café,Burger Joint,Ice Cream Shop,Japanese Restaurant,Salad Place,Bubble Tea Shop
8,Downtown Toronto,1,Coffee Shop,Café,Restaurant,Bar,Thai Restaurant,Sushi Restaurant,Seafood Restaurant,Gastropub,Lounge,Cosmetics Shop
9,Downtown Toronto,1,Coffee Shop,Aquarium,Café,Hotel,Restaurant,Brewery,Italian Restaurant,Fried Chicken Joint,Scenic Lookout,Sports Bar
10,Downtown Toronto,1,Coffee Shop,Café,Restaurant,Hotel,Bakery,Bar,Japanese Restaurant,Gastropub,Seafood Restaurant,American Restaurant


In [48]:
#Cluster 3
dt_merged.loc[dt_merged['Labels'] == 2, dt_merged.columns[[1] + list(range(5, dt_merged.shape[1]))]]

,Borough,Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Downtown Toronto,2,Airport Service,Airport Lounge,Airport Terminal,Boat or Ferry,Harbor / Marina,Sculpture Garden,Rental Car Location,Bar,Coffee Shop,Plane


In [49]:
#Cluster 4
dt_merged.loc[dt_merged['Labels'] == 3, dt_merged.columns[[1] + list(range(5, dt_merged.shape[1]))]]

,Borough,Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,Downtown Toronto,3,Grocery Store,Café,Park,Gas Station,Diner,Restaurant,Italian Restaurant,Baby Store,Candy Store,Athletics & Sports


# 3.6 Conclusion:

As seen from the above dataframes corresponding to each cluster label, the following conclusions can be made:

Cluster 1: the most common venue type is Coffee Shop, followed by restaurants and bars.<br>
Cluster 2: the most common venue type is Park or Playground.<br>
Cluster 3: the most common venue type is Airport Lounge.<br>
Cluster 4: the most common venue type is Grocery Store.<br>
The most popular venue type in Downtown Toronto is Coffee Shop, containing 16 venues total.